In [1]:
!pip install torch


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Define the neural network model
class SimpleNN(nn.Module):
    def __init__(self, input_size):
        super(SimpleNN, self).__init__()
        # One hidden layer, with an output layer
        self.hidden = nn.Linear(input_size, 5)  # Hidden layer with 5 units
        self.output = nn.Linear(5, 1)  # Output layer with 1 unit

    def forward(self, x):
        x = torch.sigmoid(self.hidden(x))  # Sigmoid activation in hidden layer
        x = torch.sigmoid(self.output(x))  # Sigmoid activation for the output
        return x

# Initialize the model with 10 inputs
input_size = 10
model = SimpleNN(input_size)


In [3]:
# Example data (assuming 10 features per row)
# Let's say we have 100 samples, each with 10 features
np.random.seed(0)
data = np.random.rand(100, 10).astype(np.float32)  # 100 samples, 10 features
labels = np.random.rand(100, 1).astype(np.float32)  # Random labels for demonstration

# Convert to PyTorch tensors
inputs = torch.tensor(data)
targets = torch.tensor(labels)


In [4]:
# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)


In [5]:
num_epochs = 1000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs, targets)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [100/1000], Loss: 0.1154
Epoch [200/1000], Loss: 0.1084
Epoch [300/1000], Loss: 0.1038
Epoch [400/1000], Loss: 0.1009
Epoch [500/1000], Loss: 0.0991
Epoch [600/1000], Loss: 0.0979
Epoch [700/1000], Loss: 0.0973
Epoch [800/1000], Loss: 0.0969
Epoch [900/1000], Loss: 0.0966
Epoch [1000/1000], Loss: 0.0965


In [6]:
# Get predictions
with torch.no_grad():
    predictions = model(inputs)

# Convert predictions to a NumPy array and rank rows
predictions = predictions.numpy().flatten()
ranked_indices = np.argsort(predictions)[::-1]  # Sort in descending order

print("Top ranked rows based on model predictions:")
print(ranked_indices[:10])  # Show the indices of top 10 ranked rows


Top ranked rows based on model predictions:
[97 54 99 83 49 34 67  5  7 88]
